In [1]:
import networkx as nx
import numpy as np

In [2]:
import mercs

In [3]:
from mercs.utils import code_to_query

# Preliminaries

Getting a very basic mercs model on my hands to test with.

In [4]:
m_codes = [
    [ 0, 0,-1,-1 ,1],
    [1,-1, 0, 0, 1],
    [-1,-1, 1,-1, 0],
    [-1,-1, 0, 1, 0]  
]

m_codes = np.array(m_codes)
m_codes

array([[ 0,  0, -1, -1,  1],
       [ 1, -1,  0,  0,  1],
       [-1, -1,  1, -1,  0],
       [-1, -1,  0,  1,  0]])

In [5]:
class model():
    def __init__(self, m_code):
        self.desc_ids = set(np.where(m_code==0)[0])
        self.targ_ids = set(np.where(m_code==1)[0])
        return

In [6]:
m_list = [model(m_code) for m_code in m_codes]
m_list[0].desc_ids

{0, 1}

In [18]:
from functools import reduce

# Algorithm

## Single-Layer Model

In [55]:
def _prune(g):
    ancestors = [nx.ancestors(g, source=n) for n in g.targ_ids]
    retain_nodes = reduce(set.union, ancestors, g.targ_ids)

    nodes_to_remove = [n for n in g.nodes if n not in retain_nodes]

    for n in nodes_to_remove:
        g.remove_node(n)
    return

def build_diagram(m_list, m_sel, q_code, g=None, prune=False):
    if g is None:
        g = nx.DiGraph()
    
    f_tgt = set([])
    a_src, a_tgt, _ = code_to_query(q_code, return_sets=True)
    
    g.desc_ids = a_src
    g.targ_ids = a_tgt
    
    for m_layer in m_sel:
        models = [(m_idx, m_list[m_idx]) for m_idx in m_layer]
        
        a_src, f_tgt, g = build_diagram_SL(models, a_src, f_tgt, g=g)
    
    if prune:
        _prune(g)
    return g

def build_diagram_SL(models, a_src, f_tgt, g=None):
    
    if g is None:
        g = nx.DiGraph()
    
    valid_src = lambda a: a in a_src
    valid_tgt = lambda a: a not in f_tgt
    
    e_src = set([])
    e_tgt = set([])
        
    for m_idx, m in models:
        for a in m.desc_ids:
            if valid_src(a):
                e_src.add((a, "M-{}".format(m_idx)))
                f_tgt.add(a)
            else:
                e_src.add(("I-{}".format(a), "M-{}".format(m_idx)))

    for m_idx, m in models:
        for a in m.targ_ids:
            if valid_tgt(a):
                e_tgt.add(("M-{}".format(m_idx), a))
                a_src.add(a)

    g.add_edges_from(e_src)
    g.add_edges_from(e_tgt)
    return a_src, f_tgt, g

In [58]:
m_sel = [np.array([0,1])]
q_code = np.array([1, 0, 0, 0, 1])


g =  build_diagram(m_list, m_sel, q_code, g=None, prune=True)
g.nodes()

{0, 4}
{0, 4}
[{2, 3, 'M-1'}, {1, 2, 3, 'M-1', 'M-0', 'I-0'}]
[]


NodeView((3, 'M-1', 'I-0', 'M-0', 1, 2, 0, 4))

In [60]:
list(nx.topological_sort(g))

[2, 1, 'I-0', 'M-0', 3, 'M-1', 4, 0]

Change inputs in dask is still absolutely crucial. But this network way of viewing ML as a whole could be a very, very cool idea. Especially if you want the problem to be treated recursively. Please persist in your efforts. In the end, the idea would be to do ML on an unprecendented scale, decentralized yet coordinated.

# Inference